In [1]:
#Importing the necessary libraries
import yfinance as yf
import pandas as pd
import requests
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
#Reading the dataset
df=pd.read_csv('Data.csv', encoding = "ISO-8859-1")

In [3]:
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000-01-03,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,...,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,2000-01-04,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,...,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite
2,2000-01-05,0,Coventry caught on counter by Flo,United's rivals on the road to Rio,Thatcher issues defence before trial by video,Police help Smith lay down the law at Everton,Tale of Trautmann bears two more retellings,England on the rack,Pakistan retaliate with call for video of Walsh,Cullinan continues his Cape monopoly,...,South Melbourne (Australia),Necaxa (Mexico),Real Madrid (Spain),Raja Casablanca (Morocco),Corinthians (Brazil),Tony's pet project,Al Nassr (Saudi Arabia),Ideal Holmes show,Pinochet leaves hospital after tests,Useful links
3,2000-01-06,1,Pilgrim knows how to progress,Thatcher facing ban,McIlroy calls for Irish fighting spirit,Leicester bin stadium blueprint,United braced for Mexican wave,"Auntie back in fashion, even if the dress look...",Shoaib appeal goes to the top,Hussain hurt by 'shambles' but lays blame on e...,...,Putin admits Yeltsin quit to give him a head s...,BBC worst hit as digital TV begins to bite,How much can you pay for...,Christmas glitches,"Upending a table, Chopping a line and Scoring ...","Scientific evidence 'unreliable', defence claims",Fusco wins judicial review in extradition case,Rebels thwart Russian advance,Blair orders shake-up of failing NHS,Lessons of law's hard heart
4,2000-01-07,1,Hitches and Horlocks,Beckham off but United survive,Breast cancer screening,Alan Parker,Guardian readers: are you all whingers?,Hollywood Beyond,Ashes and diamonds,Whingers - a formidable minority,...,Most everywhere: UDIs,Most wanted: Chloe lunettes,Return of the cane 'completely off the agenda',From Sleepy Hollow to Greeneland,Blunkett outlines vision for over 11s,"Embattled Dobson attacks 'play now, pay later'...",Doom and the Dome,What is the north-south divide?,Aitken released from jail,Gone aloft


In [4]:
#Dividing the data into testing and training set
train = df[df['Date'] < '20150101']
test = df[df['Date'] > '20141231']

In [5]:
# Removing punctuations from data
data=train.iloc[:,2:27]
data.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

# Renaming column names for ease of access
list1= [i for i in range(25)]
new_Index=[str(i) for i in list1]
data.columns= new_Index
data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,A hindrance to operations extracts from the...,Scorecard,Hughes instant hit buoys Blues,Jack gets his skates on at ice cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,Derby raise a glass to Strupar s debut double,Southgate strikes Leeds pay the penalty,...,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl s successor drawn into scandal,The difference between men and women,Sara Denver nurse turned solicitor,Diana s landmine crusade put Tories in a panic,Yeltsin s resignation caught opposition flat f...,Russian roulette,Sold out,Recovering a title
1,Scorecard,The best lake scene,Leader German sleaze inquiry,Cheerio boyo,The main recommendations,Has Cubie killed fees,Has Cubie killed fees,Has Cubie killed fees,Hopkins furious at Foster s lack of Hannibal...,Has Cubie killed fees,...,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man s extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn t know without the ...,Millennium bug fails to bite
2,Coventry caught on counter by Flo,United s rivals on the road to Rio,Thatcher issues defence before trial by video,Police help Smith lay down the law at Everton,Tale of Trautmann bears two more retellings,England on the rack,Pakistan retaliate with call for video of Walsh,Cullinan continues his Cape monopoly,McGrath puts India out of their misery,Blair Witch bandwagon rolls on,...,South Melbourne Australia,Necaxa Mexico,Real Madrid Spain,Raja Casablanca Morocco,Corinthians Brazil,Tony s pet project,Al Nassr Saudi Arabia,Ideal Holmes show,Pinochet leaves hospital after tests,Useful links
3,Pilgrim knows how to progress,Thatcher facing ban,McIlroy calls for Irish fighting spirit,Leicester bin stadium blueprint,United braced for Mexican wave,Auntie back in fashion even if the dress look...,Shoaib appeal goes to the top,Hussain hurt by shambles but lays blame on e...,England s decade of disasters,Revenge is sweet for jubilant Cronje,...,Putin admits Yeltsin quit to give him a head s...,BBC worst hit as digital TV begins to bite,How much can you pay for,Christmas glitches,Upending a table Chopping a line and Scoring ...,Scientific evidence unreliable defence claims,Fusco wins judicial review in extradition case,Rebels thwart Russian advance,Blair orders shake up of failing NHS,Lessons of law s hard heart
4,Hitches and Horlocks,Beckham off but United survive,Breast cancer screening,Alan Parker,Guardian readers are you all whingers,Hollywood Beyond,Ashes and diamonds,Whingers a formidable minority,Alan Parker part two,Thuggery Toxins and Ties,...,Most everywhere UDIs,Most wanted Chloe lunettes,Return of the cane completely off the agenda,From Sleepy Hollow to Greeneland,Blunkett outlines vision for over s,Embattled Dobson attacks play now pay later ...,Doom and the Dome,What is the north south divide,Aitken released from jail,Gone aloft


In [6]:
# Convertng headlines to lower case
for index in new_Index:
    data[index]=data[index].str.lower()
data.head(1)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,a hindrance to operations extracts from the...,scorecard,hughes instant hit buoys blues,jack gets his skates on at ice cold alex,chaos as maracana builds up for united,depleted leicester prevail as elliott spoils e...,hungry spurs sense rich pickings,gunners so wide of an easy target,derby raise a glass to strupar s debut double,southgate strikes leeds pay the penalty,...,flintoff injury piles on woe for england,hunters threaten jospin with new battle of the...,kohl s successor drawn into scandal,the difference between men and women,sara denver nurse turned solicitor,diana s landmine crusade put tories in a panic,yeltsin s resignation caught opposition flat f...,russian roulette,sold out,recovering a title


In [7]:
headlines = []
for row in range(0,len(data.index)):
    headlines.append(' '.join(str(x) for x in data.iloc[row,0:25]))

In [8]:
headlines[0]

'a  hindrance to operations   extracts from the leaked reports scorecard hughes  instant hit buoys blues jack gets his skates on at ice cold alex chaos as maracana builds up for united depleted leicester prevail as elliott spoils everton s party hungry spurs sense rich pickings gunners so wide of an easy target derby raise a glass to strupar s debut double southgate strikes  leeds pay the penalty hammers hand robson a youthful lesson saints party like it s      wear wolves have turned into lambs stump mike catches testy gough s taunt langer escapes to hit     flintoff injury piles on woe for england hunters threaten jospin with new battle of the somme kohl s successor drawn into scandal the difference between men and women sara denver  nurse turned solicitor diana s landmine crusade put tories in a panic yeltsin s resignation caught opposition flat footed russian roulette sold out recovering a title'

In [9]:
#Applying CountVectorixer and ngram model of 2
countvector=CountVectorizer(ngram_range=(2,2))
traindataset=countvector.fit_transform(headlines)

In [10]:
#Fitting the model using Naive Bayes Classifier
classifier = MultinomialNB().fit(traindataset,train['Label'])

In [11]:
#Applying same transformations on the test data and predicting results
test_transform= []
for row in range(0,len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row,2:27]))
test_dataset = countvector.transform(test_transform)
predictions = classifier.predict(test_dataset)

In [12]:
#Constructing Confusion matrix and calculating accuracy score 
matrix=confusion_matrix(test['Label'],predictions)
print(matrix)
score=accuracy_score(test['Label'],predictions)
print(score)
report=classification_report(test['Label'],predictions)
print(report)

[[138  48]
 [ 10 182]]
0.8465608465608465
              precision    recall  f1-score   support

           0       0.93      0.74      0.83       186
           1       0.79      0.95      0.86       192

    accuracy                           0.85       378
   macro avg       0.86      0.84      0.84       378
weighted avg       0.86      0.85      0.84       378



#### Fetching a sample data from Yahoo API

In [13]:
url = "https://yfapi.net/ws/insights/v1/finance/insights"

querystring = {"symbol":"AAPL"}

headers = {
    'x-api-key': "8noO5BUSZ92mcTXq2p5tD861wPHPhCqx5nVuEyEL"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"finance":{"result":{"symbol":"AAPL","instrumentInfo":{"technicalEvents":{"provider":"Trading Central","shortTerm":"down","midTerm":"down","longTerm":"up"},"keyTechnicals":{"provider":"Trading Central","support":149.57,"resistance":172.21,"stopLoss":155.005718},"valuation":{"color":0.0,"description":"Overvalued","discount":"12%","relativeValue":"Premium","provider":"Trading Central"},"recommendation":{"targetPrice":180.0,"provider":"Argus Research","rating":"BUY"}},"reports":[{"id":"32519_Market Outlook_1648631574000","title":"Daily Spotlight: Final 4Q GDP Reading: +6.9%","provider":"Argus Research","publishedOn":"2022-03-30T09:12:54Z","summary":"This morning, the U.S. Department of Commerce announced that its \"third\" estimate for the rate of growth in 4Q21 GDP was 6.9%. This was slightly below the previous estimates but in line with the consensus outlook. In our estimation, this will be the last time for a long time that the country experiences GDP growth in the 7.0% range. The 4Q 

In [14]:
#This is the news summary that we need to predict
data = response.json()
print(data['finance']['result']['reports'][10]['summary'])

Apple manufactures smartphones, tablets, P6Cs, software and peripherals for a worldwide customer base. Its products include Mac desktop and mobile PCs, iPhone, iPad, Apple Watch, and various consumer products, including AirPods, Beats headphones and Apple TV. Apple services include App Store, iTunes, iCloud, Apple TV+, Apple Arcade, Apple Music, Apple Pay, and more.


In [15]:
test_df = pd.DataFrame()

In [16]:
#Making a nested list of different summaries 
lst=[]
for i in range(25):
    text = [data['finance']['result']['reports'][i]['summary']]
    lst.append(text)

In [17]:
print(lst)

[['This morning, the U.S. Department of Commerce announced that its "third" estimate for the rate of growth in 4Q21 GDP was 6.9%. This was slightly below the previous estimates but in line with the consensus outlook. In our estimation, this will be the last time for a long time that the country experiences GDP growth in the 7.0% range. The 4Q period was post-Delta and pre-Omicron. In addition, since the end of the fourth quarter, Russia has invaded Ukraine and the Federal Reserve has started to raise short-term interest rates. These factors, as well as the inevitable emergence of new COVID strains, are likely to send GDP growth back down to the pre-pandemic level of 2.0%-3.0%. The GDP report also includes an inflation gauge, the PCE price index. This index, excluding food and energy, rose at a 5.0% pace, well ahead the Federal Reserve\'s inflation target of 2.0% and above the 4.6% rate recorded in 3Q21. Looking ahead, our forecast for 2022 GDP growth is now 3.1%, compared to our estima

In [18]:
#Making a dataframe by combining different summaries 
for i in range(25):
    text = lst[i]
    test_df[i]=text

In [19]:
test_df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,"This morning, the U.S. Department of Commerce ...","Apple manufactures smartphones, tablets, P6Cs,...",Stocks finished lower on Thursday amid continu...,"Apple manufactures smartphones, tablets, P6Cs,...","Stocks rose strongly on Friday morning, led by...",The U.S. Department of Commerce announced that...,Sustainable Impact Investing is gaining tracti...,"Apple manufactures smartphones, tablets, P6Cs,...",Stocks rose strongly on Thursday as investors ...,The United States economy is full of innovatio...,...,"In the trading week ended September 10, the st...",The Technology sector started slowly in 2021 a...,A lot of attention these days is being given t...,July's stock-market performance was positive a...,"Apple manufactures smartphones, tablets, PCs, ...",Stocks were mixed on Wednesday morning as inve...,The U.S. Department of Commerce last week anno...,The U.S. Department of Commerce last week anno...,"In our Wednesday comment, we noted that we don...","Apple manufactures smartphones, tablets, PCs, ..."


In [20]:
test_df.shape

(1, 25)

In [21]:
#Applying some preprocessing on the collected data and predicting the results
test_transform= []
for row in range(0,len(test_df.index)):
    test_transform.append(' '.join(str(x) for x in test_df.iloc[row,0:24]))
test_dataset = countvector.transform(test_transform)
predictions = classifier.predict(test_dataset)

In [22]:
print(predictions)

[1]


#### We can see that the prediction is 1, hence it is beneficial to purchse the stock